<a href="https://colab.research.google.com/github/dkle91/Hyperparameter_Optimization_for_PINNs_using_GA/blob/main/Diffusion_equation_GA_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
from sklearn import preprocessing
from google.colab import files

# Import result file
file = open("Diffusion-data.csv")
result = np.loadtxt(file, delimiter=",")
#print(Diffusion_result)
# Define parameter
population_size = 100
w_time = 0
w_loss = 1
tournament_size = 2
l = 8 # number of hyperparameter + 1
missing_value = [999999]
mutation_rate = 0.2 # posibility of mutation 
num_generation = 500
total_population = np.empty((num_generation+1, population_size, l+3))
total_best = np.empty((num_generation+1, l+3))
desired_error = 2e-7

#collapse-show
def create_population(population_size):
    ini_population = np.empty((population_size,l+3))
    indicator = random.sample(range(0, result.shape[0]+1), population_size)
    indicator = np.array(indicator)
    for m in range(population_size):
      ini_population[m] =result[indicator[m]] 
    return ini_population

def problem(population, soln):
    normalize = np.concatenate((w_time*population[:,l],w_loss*population[:,l+1]), axis=None)
    normalized = preprocessing.normalize(normalize.reshape(1,-1))
    normalized = normalized.reshape(-1,1)
    normalized_time, normalized_loss = normalized[:len(population)], normalized[len(population):]
    return normalized_time[soln]+normalized_loss[soln]
     
def assess_fitness(individual,population, problem):
    "Determines the fitness of an individual using the given problem"
    return problem(population, individual)

def find_current_best(population, problem):
    """Evaluates a given population and returns the fittest individual.
    This can be sped up to only loop over popuation once, but because this is a tutorial, 3 lines is nicer.
    """
    fitnesses = [assess_fitness(x, population, problem) for x in range(len(population))]
    best_value = min(fitnesses) # Lowest is best
    best_index = fitnesses.index(best_value)
    return population[best_index]

# Next generation
def tournament_select_with_replacement(population, tournament_size, problem):
    "Competes a number of challengers and returns the fittest one"
    challengers_indexes = np.random.choice(population.shape[0], tournament_size, replace=True)
    challengers = population[challengers_indexes]
    return find_current_best(challengers, problem)
     
#collapse-show
def crossover(parent_a, parent_b):
    "Performs two point crossover on two parents"
    c, d = random.randint(1, 5), random.randint(5, l)
    temp = np.copy(parent_a)
    child_a = np.concatenate([missing_value,parent_a[1:c], parent_b[c:d], parent_a[d:l],missing_value,missing_value, missing_value])
    child_b = np.concatenate([missing_value,parent_b[1:c], temp[c:d], parent_b[d:l],missing_value,missing_value, missing_value])
    return child_a, child_b

def mutate(child, mutation_rate):
    "May mutate a child using Gaussian convolution"
    if mutation_rate >= random.uniform(0,1):
      for j in range(l-1):
        child[j+1] = np.random.randint(0,3)
    for i in range(len(result)):
      if (child[1]==result[i,1]) and (child[2]==result[i,2]) and (child[3]==result[i,3]) and (child[4]==result[i,4]) and (child[5]==result[i,5]) and (child[6]==result[i,6]) and (child[7]==result[i,7]):
        child[0] = result[i,0]
        child[8] = result[i,8]
        child[9] = result[i,9]
        child[10] = result[i,10] 
    return child

def update_population(population, problem, mutation_rate):
    """Performs one generational update of Genetic Algorithm"""
    #pop_size = len(population)
    next_population = np.empty((population_size, l+3))
    for k in range(int(population_size / 2)):
        parent_a = tournament_select_with_replacement(population, tournament_size, problem)
        parent_b = tournament_select_with_replacement(population, tournament_size, problem)
        child_a, child_b = crossover(parent_a, parent_b)
        next_population[k] = mutate(child_a, mutation_rate)
        position_child_b = k + (population_size / 2)
        next_population[int(position_child_b)] = mutate(child_b, mutation_rate)
    return next_population

total_population[0] = create_population(population_size)
#total_population[0] = current_population
total_best[0] = find_current_best(total_population[0], problem)
fig = plt.figure(figsize=(9,6))
plt.scatter(total_population[0,:,l], total_population[0,:,l+1], c='blue', marker='.',alpha=1)
plt.scatter(total_best[0,l], total_best[0,l+1], c='red', marker='x',alpha=1)
plt.gca().legend(('Population','Fittest'))
plt.xlabel('Computational time (s)')
plt.ylabel('Loss')
plt.title('Genetic Algoritm: Initial population_100-20_diffusion equation.jpeg')
plt.savefig('Genetic Algoritm: Initial population_100-20_diffusion equation.jpeg', bbox_inches='tight', dpi=300)
#files.download('Genetic Algoritm: Initial population_100-20_diffusion equation.jpeg') 
print('The best hyperparameter of Initial population is: ', total_best[0])

from google.colab import files
i=0
stop = 9999
last_best = total_best[0,l+1]
while ((stop != last_best) or (stop >= desired_error)):
  total_population[i+1] = update_population(total_population[i], problem, mutation_rate)
  total_best[i+1] = find_current_best(total_population[i+1], problem)
  fig = plt.figure(figsize=(9,6))
  plt.scatter(total_population[i+1,:,l], total_population[i+1,:,l+1], c='blue', marker='.',alpha=1)
  plt.scatter(total_best[i+1,l], total_best[i+1,l+1], c='red', marker='x',alpha=1)
  plt.gca().legend(('Population','Fittest'))
  plt.xlabel('Computational time (s)')
  plt.ylabel('Loss')
  if i ==0:
    plt.title('Genetic Algorithm: The %dnd generation'%(i+2))
    plt.savefig('Genetic Algorithm: The %dnd generation_100-20_diffusion equation.jpeg'%(i+2), bbox_inches='tight', dpi=300)
    #files.download('Genetic Algorithm: The %dnd generation_100-20_diffusion equation.jpeg'%(i+2)) 
  else:
    plt.title('Genetic Algorithm: The %dth generation'%(i+2))
    plt.savefig('Genetic Algorithm: The %dth generation_100-20_diffusion equation.jpeg'%(i+2), bbox_inches='tight', dpi=300)
    #files.download('Genetic Algorithm: The %dth generation_100-20_diffusion equation.jpeg'%(i+2)) 
  print('The best hyperparameter of the %dth population is: '%(i+2), total_best[i+1])
  stop = total_best[i+1,l+1]
  last_best = total_best[i,l+1]
  i+=1
lentotal = i

#for i in range(lentotal+1):
  #np.savetxt('Genetic Algorithm: The %dth population_100-20_diffusion equation.csv'%i, total_population[i], delimiter=',')
  #files.download('Genetic Algorithm: The %dth population_100-20_diffusion equation.csv'%i) 
#np.savetxt('Genetic Algorithm: Fittest in all generation_100-20_diffusion equation.csv', total_best, delimiter=',')
#files.download('Genetic Algorithm: Fittest in all generation_100-20_diffusion equation.csv') 

total_time = 0
for i in range(lentotal+1):
  total_time += np.sum(total_population[i,:,l])
print('The total time is (s):', total_time) 
